# Copernicus Climate Data Store (CDS) API
- CDS documentation: https://confluence.ecmwf.int/display/CKB/Climate+Data+Store+%28CDS%29+documentation
- Developer and API documentation: https://cds.climate.copernicus.eu/toolbox/doc/index.html
- Use bboxfinder tool to select a suitable geographic bounding box: http://bboxfinder.com

In [1]:
from pathlib import Path

import cdsapi
import numpy as np
import pandas as pd

In [12]:
# Directory where data files will be downloaded
cwd_path = Path.cwd()
data_path = cwd_path.parent / 'data'

In [13]:
data_path

PosixPath('/Users/brand/my_code/meteoviz/course_project/data')

In [14]:
# Make sure api key is in dot file in home dir
cds = cdsapi.Client()
print(cds.url)
print(cds.verify)
print(cds.timeout)

https://cds.climate.copernicus.eu/api/v2
True
60


#### Forming the API request
- From the documentation: "We strongly suggest to construct CDS API requests by using the CDS web interface of the relevant dataset and using the 'Show API request' button to get the code."

#### Suitable geographic bounding box that contains all of Germany
- Box coordinates (long,lat): 4.218,46.073,17.028,56.377
- CDS needs it in North(0 to 90), West(0 to -180), South(0 to -90), East (0 to 180). Default: global
    - That's `[56.38, 4.22, 46.07, 17.03]`

#### Data size and request duration
- One year of hourly 100m and 10m wind components (4 variables) for above bounding box is 150 MB
- API call took 17 minutes from queued, running to completed

#### Variables of interest
- `100m_u-component_of_wind`, `100m_v-component_of_wind` and 10m components have units `m s**-1` and  are single level, instantanous parameters
- instantaneous: Although they are valid at the specified time, and NOT an average over the model time step, such parameters cannot represent variability on short time scales. This arises because the model values are a grid box average, without variability on spatial scales smaller than the grid

In [22]:
# # Get hourly 10m and 100m wind components for one year
# cds.retrieve(
#     'reanalysis-era5-single-levels',
#     {
#         'product_type': 'reanalysis',
#         'format': 'netcdf',
#         'variable': [
#             '100m_u_component_of_wind', '100m_v_component_of_wind', '10m_u_component_of_wind',
#             '10m_v_component_of_wind',
#         ],
#         'year': '2022',
#         'month': [
#             '01', '02', '03',
#             '04', '05', '06',
#             '07', '08', '09',
#             '10', '11', '12',
#         ],
#         'day': [
#             '01', '02', '03',
#             '04', '05', '06',
#             '07', '08', '09',
#             '10', '11', '12',
#             '13', '14', '15',
#             '16', '17', '18',
#             '19', '20', '21',
#             '22', '23', '24',
#             '25', '26', '27',
#             '28', '29', '30',
#             '31',
#         ],
#         'time': [
#             '00:00', '01:00', '02:00',
#             '03:00', '04:00', '05:00',
#             '06:00', '07:00', '08:00',
#             '09:00', '10:00', '11:00',
#             '12:00', '13:00', '14:00',
#             '15:00', '16:00', '17:00',
#             '18:00', '19:00', '20:00',
#             '21:00', '22:00', '23:00',
#         ],
#         'area': [
#             56.38, 4.22, 46.07,
#             17.03,
#         ],
#     },
#     str(data_path.joinpath('wind_data.nc')))

2023-08-27 19:45:59,951 INFO Welcome to the CDS
2023-08-27 19:45:59,952 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-08-27 19:46:00,046 INFO Request is queued
2023-08-27 19:46:01,127 INFO Request is running
2023-08-27 20:04:21,292 INFO Request is completed
2023-08-27 20:04:21,293 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data6/adaptor.mars.internal-1693159264.857704-31102-17-b5d2bf2f-fdcd-4ae0-9805-31aef5dfb1fd.nc to /Users/brand/my_code/meteoviz/course_project/data/wind_data.nc (146M)
2023-08-27 20:04:51,669 INFO Download rate 4.8M/s                             


Result(content_length=153091976,content_type=application/x-netcdf,location=https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data6/adaptor.mars.internal-1693159264.857704-31102-17-b5d2bf2f-fdcd-4ae0-9805-31aef5dfb1fd.nc)

#### Note
- Is the projection of the grid: equal-area EPSG:3035 projection? 
- Grid resolution should be 30km x 30km
    - ERA5 data is referenced in the horizontal with respect to the WGS84 ellipse (which defines the major/minor axes) and in the vertical it is referenced to the EGM96 geoid over land but over ocean it is referenced to mean sea level, with the approximation that this is assumed to be coincident with the geoid
    - The native grid is the reduced Gaussian grid N320

#### Reading and writing netCDF files in Xarray
- https://docs.xarray.dev/en/stable/user-guide/io.html